In [1]:
from PIL import Image,ImageOps
import numpy as np
import cv2
import openslide
import pyvips
from glob import glob
import os
from tqdm import tqdm

In [ ]:
slide_list=['../../data/BR_HnE/NIPA-STOP-KN-00041-S-TP-01.ndpi']
for i in tqdm(range(len(slide_list))):

    slide = pyvips.Image.new_from_file(slide_list[i], access='sequential')
    src_img=Image.open(f'../../results/{os.path.basename(slide_list[i]).split('.')[0]}.png')
    dst_img=ImageOps.expand(src_img, border=(0, 0, (slide.width-src_img.size[0]*32)//32, (slide.height-src_img.size[1]*32)//32), fill=(255, 255, 255))
    
    # 투명도 설정을 위해 RGBA로 변환
    dst_img = dst_img.convert('RGBA')
    data = np.array(dst_img)
    
    # 흰색 픽셀 찾기 (RGB 값이 모두 255인 경우)
    white_mask = (data[:,:,0] == 255) & (data[:,:,1] == 255) & (data[:,:,2] == 255)
    
    # 흰색은 완전 투명 (알파값 0), 나머지는 50% 투명 (알파값 128)
    data[white_mask, 3] = 0     # 완전 투명
    data[~white_mask, 3] = 75  # 50% 투명
    
    # numpy 배열을 다시 PIL 이미지로 변환
    dst_img = Image.fromarray(data, 'RGBA')
    dst_img.save(f'../../results/{os.path.basename(slide_list[i]).split(".")[0]}_seg_padding.png')
    # OpenCV - 빠름
    

100%|██████████| 1/1 [00:01<00:00,  1.18s/it]


In [21]:
slide.width

161280